In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
import mysql_conn
import datetime

In [2]:
#Read all the fuel economy data for smartway vehicles 
#year: 2016,2017,2018
fuel_2016_path = 'Resources/all_alpha_16.xlsx'
fuel_2017_path = 'Resources/all_alpha_17.xlsx'
fuel_2018_path = 'Resources/all_alpha_18.xlsx'

all_fuel_2016_df = pd.read_excel(fuel_2016_path)
all_fuel_2017_df = pd.read_excel(fuel_2017_path)
all_fuel_2018_df = pd.read_excel(fuel_2018_path)

In [3]:
#check the dataset
all_fuel_2016_df.head()

,Model,Displ,Cyl,Trans,Drive,Fuel,Cert Region,Stnd,Stnd Description,Underhood ID,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,SmartWay,Comb CO2
0,ACURA ILX,2.4,4.0,AutoMan-8,2WD,Gasoline,FA,B5,Federal Tier 2 Bin 5,GHNXV02.4XH3,small car,5,25,36,29,7,No,305
1,ACURA ILX,2.4,4.0,AutoMan-8,2WD,Gasoline,CA,L3ULEV125,California LEV-III ULEV125,GHNXV02.4XH3,small car,6,25,36,29,7,Yes,305
2,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,FA,B5,Federal Tier 2 Bin 5,GHNXV03.5VA3,small SUV,5,19,27,22,5,No,403
3,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,FA,B5,Federal Tier 2 Bin 5,GHNXV03.5VA3,small SUV,5,20,27,23,5,No,390
4,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,CA,U2,California LEV-II ULEV,GHNXV03.5VA3,small SUV,6,19,27,22,5,No,403


In [18]:
def manage_df(all_fuel_df,year):
            
    select_df = all_fuel_df.drop(
        [ 'Cert Region', 'Stnd', 'Stnd Description', 'Underhood ID'], axis=1)
    fuel_df = select_df.rename(columns={
                                     'Model': 'model', 'Veh Class': 'vehicle_class',
                                    'Displ':'displ','Cyl':'cyl','Trans':'trans','Drive':'drive',
                                     'Fuel': 'fuel_type', 'Air Pollution Score': 'smog_rating', 'City MPG': 'city_mpg'
                                    , 'Hwy MPG': 'hwy_mpg','Cmb MPG': 'cmb_mpg', 
                                    'Greenhouse Gas Score':'Greenhouse_gas_score',
                                    'SmartWay':'smartway','Comb CO2':'comb_CO2'}, index=str)
    fuel_df['year'] = year
    return fuel_df

In [19]:
fuel_2016_df = manage_df(all_fuel_2016_df,2016)
fuel_2017_df = manage_df(all_fuel_2017_df,2017)
fuel_2018_df = manage_df(all_fuel_2018_df,2018)

In [20]:
#Check the datatype of all the columns
fuel_2016_df.dtypes

model                    object
displ                   float64
cyl                     float64
trans                    object
drive                    object
fuel_type                object
vehicle_class            object
smog_rating              object
city_mpg                 object
hwy_mpg                  object
cmb_mpg                  object
Greenhouse_gas_score     object
smartway                 object
comb_CO2                 object
year                      int64
dtype: object

In [21]:
fuel_2016_df.head()

,model,displ,cyl,trans,drive,fuel_type,vehicle_class,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,year
0,ACURA ILX,2.4,4.0,AutoMan-8,2WD,Gasoline,small car,5,25,36,29,7,No,305,2016
1,ACURA ILX,2.4,4.0,AutoMan-8,2WD,Gasoline,small car,6,25,36,29,7,Yes,305,2016
2,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,small SUV,5,19,27,22,5,No,403,2016
3,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,small SUV,5,20,27,23,5,No,390,2016
4,ACURA MDX,3.5,6.0,SemiAuto-9,2WD,Gasoline,small SUV,6,19,27,22,5,No,403,2016


model VARCHAR(50),
vehicle_class VARCHAR(50),
fuel_type VARCHAR(50),
smog_rating  INT,
city_mpg   INT,
hwy_mpg   INT,
cmb_mpg  INT,
Greenhouse_gas_score  INT,
smartway  VARCHAR(50)

In [22]:
#function: to covert the two nos from string to interger and calculate the average 
def average(number1, number2):
    return (int(number1) + int(number2)) / 2.0

In [42]:
#Function: Conver the mpg values to float and if it is hybrid car the take the average of gasoline and electric
def convert_mpg(mpg_value):
    mpg_list = []

    for i in range (len(mpg_value)):
        if '/' in (mpg_value[i]):
            mpg = mpg_value[i].split('/')
            avr_mpg = average(mpg[0],mpg[1])
            mpg_list.append(avr_mpg)
        else:
            mpg_row = float(mpg_value[i])
            mpg_list.append(mpg_row)
    return mpg_list
   
   

In [43]:
#Call convert_mpg function for all mpg columns of all 3 years
city_2016_list = convert_mpg(fuel_2016_df['city_mpg'])
hwy_2016_list = convert_mpg(fuel_2016_df['hwy_mpg'])
cmb_2016_list = convert_mpg(fuel_2016_df['cmb_mpg'])

city_2017_list = convert_mpg(fuel_2017_df['city_mpg'])
hwy_2017_list = convert_mpg(fuel_2017_df['hwy_mpg'])
cmb_2017_list = convert_mpg(fuel_2017_df['cmb_mpg'])

city_2018_list = convert_mpg(fuel_2018_df['city_mpg'])
hwy_2018_list = convert_mpg(fuel_2018_df['hwy_mpg'])
cmb_2018_list = convert_mpg(fuel_2018_df['cmb_mpg'])

comb_CO2_2016_list = convert_mpg(fuel_2016_df['comb_CO2'])
comb_CO2_2017_list = convert_mpg(fuel_2017_df['comb_CO2'])
comb_CO2_2018_list = convert_mpg(fuel_2018_df['comb_CO2'])

In [46]:
def replace_all_mpg_col(fuel_mpg_df,city_mpg_list,hwy_mpg_list,cmb_mpg_list,comb_CO2_list):
    # Drop that column
    select_mpg_df = fuel_mpg_df.drop(
        ['city_mpg', 'hwy_mpg', 'cmb_mpg','comb_CO2'], axis=1)

    # Put all 3 series  in its place
    select_mpg_df['city_mpg'] = city_mpg_list
    select_mpg_df['hwy_mpg'] = hwy_mpg_list
    select_mpg_df['cmb_mpg'] = cmb_mpg_list
    select_mpg_df['comb_CO2'] = comb_CO2_list
        #redesing the columns position 

    new_mpg_df = select_mpg_df[['year','model','fuel_type','vehicle_class','displ','cyl','trans','drive','smog_rating','city_mpg','hwy_mpg',
             'cmb_mpg','Greenhouse_gas_score','smartway','comb_CO2']]
    return new_mpg_df
    

In [47]:
new_city_mpg_2016_df = replace_all_mpg_col(fuel_2016_df,city_2016_list,
                                           hwy_2016_list,cmb_2016_list,comb_CO2_2016_list)
new_city_mpg_2016_df.head()

,year,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2
0,2016,ACURA ILX,Gasoline,small car,2.4,4.0,AutoMan-8,2WD,5,25.0,36.0,29.0,7,No,305.0
1,2016,ACURA ILX,Gasoline,small car,2.4,4.0,AutoMan-8,2WD,6,25.0,36.0,29.0,7,Yes,305.0
2,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,5,19.0,27.0,22.0,5,No,403.0
3,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,5,20.0,27.0,23.0,5,No,390.0
4,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,6,19.0,27.0,22.0,5,No,403.0


In [48]:
new_city_mpg_2017_df = replace_all_mpg_col(fuel_2017_df,city_2017_list,
                                           hwy_2017_list,cmb_2017_list,comb_CO2_2017_list)
new_city_mpg_2017_df.head()

,year,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2
0,2017,ACURA ILX,Gasoline,small car,2.4,4.0,AMS-8,2WD,6,25.0,35.0,29.0,7,Yes,309.0
1,2017,ACURA ILX,Gasoline,small car,2.4,4.0,AMS-8,2WD,6,25.0,35.0,29.0,7,Yes,309.0
2,2017,ACURA MDX,Gasoline,small SUV,3.0,6.0,AMS-7,4WD,6,26.0,27.0,27.0,6,No,329.0
3,2017,ACURA MDX,Gasoline,small SUV,3.0,6.0,AMS-7,4WD,6,26.0,27.0,27.0,6,No,329.0
4,2017,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,6,19.0,27.0,22.0,5,No,404.0


In [49]:
new_city_mpg_2018_df = replace_all_mpg_col(fuel_2018_df,
                                           city_2018_list,hwy_2018_list,cmb_2018_list,comb_CO2_2018_list)
new_city_mpg_2018_df.head()

,year,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2
0,2018,ACURA ILX,Gasoline,small car,2.4,4.0,AMS-8,2WD,3,25.0,35.0,29.0,6,No,309.0
1,2018,ACURA ILX,Gasoline,small car,2.4,4.0,AMS-8,2WD,3,25.0,35.0,29.0,6,No,309.0
2,2018,ACURA MDX,Gasoline,small SUV,3.0,6.0,AMS-7,4WD,3,26.0,27.0,27.0,6,No,330.0
3,2018,ACURA MDX,Gasoline,small SUV,3.0,6.0,AMS-7,4WD,3,26.0,27.0,27.0,6,No,330.0
4,2018,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,3,19.0,27.0,22.0,4,No,404.0


In [50]:
final_mpg_df = pd.concat([new_city_mpg_2016_df, new_city_mpg_2017_df,
                          new_city_mpg_2018_df], axis=0).reset_index(drop=True)
final_mpg_df.index.name = 'id'

In [51]:
final_mpg_df.head()

,year,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2
id,,,,,,,,,,,,,,,
0,2016,ACURA ILX,Gasoline,small car,2.4,4.0,AutoMan-8,2WD,5,25.0,36.0,29.0,7,No,305.0
1,2016,ACURA ILX,Gasoline,small car,2.4,4.0,AutoMan-8,2WD,6,25.0,36.0,29.0,7,Yes,305.0
2,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,5,19.0,27.0,22.0,5,No,403.0
3,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,5,20.0,27.0,23.0,5,No,390.0
4,2016,ACURA MDX,Gasoline,small SUV,3.5,6.0,SemiAuto-9,2WD,6,19.0,27.0,22.0,5,No,403.0


In [52]:
final_mpg_df.tail()

,year,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2
id,,,,,,,,,,,,,,,
7835,2018,VOLVO XC 90,Gasoline,standard SUV,2.0,4.0,SemiAuto-8,4WD,5,22.0,28.0,24.0,5,No,365.0
7836,2018,VOLVO XC 90,Gasoline,standard SUV,2.0,4.0,SemiAuto-8,4WD,5,20.0,27.0,23.0,5,No,394.0
7837,2018,VOLVO XC 90,Gasoline,standard SUV,2.0,4.0,SemiAuto-8,4WD,5,20.0,27.0,23.0,5,No,394.0
7838,2018,VOLVO XC 90,Gasoline/Electricity,standard SUV,2.0,4.0,SemiAuto-8,4WD,7,44.5,45.5,44.5,10,Elite,187.0
7839,2018,VOLVO XC 90,Gasoline/Electricity,standard SUV,2.0,4.0,SemiAuto-8,4WD,7,44.5,45.5,44.5,10,Elite,187.0


In [53]:
print(len(new_city_mpg_2016_df))
print(len(new_city_mpg_2017_df))
print(len(new_city_mpg_2018_df))
print(len(final_mpg_df))
file_name = 'Resources/final_all_data.csv'
final_mpg_df.to_csv(file_name)


2585
2573
2682
7840


In [54]:
final_mpg_df.isna().any()

year                    False
model                   False
fuel_type               False
vehicle_class           False
displ                    True
cyl                      True
trans                   False
drive                   False
smog_rating             False
city_mpg                False
hwy_mpg                 False
cmb_mpg                 False
Greenhouse_gas_score    False
smartway                False
comb_CO2                False
dtype: bool

### Create database connection

In [ ]:
# Create Engine and Pass in MySQL Connection
connection_string = (
    f"root:{mysql_conn.password}@localhost/fuel_economy_db")
engine = create_engine(f'mysql://{connection_string}')


In [ ]:
# Confirm tables
engine.table_names()


In [ ]:
pd.read_sql_query('select * from Fuel', con=engine).head()

### Load DataFrames into database

In [ ]:
Base = declarative_base()
session = Session(bind=engine)

In [ ]:
# Object relational mapping for our table
# ----------------------------------
class Fuel(Base):
    __tablename__ = 'fuel'
    id = Column(Integer, primary_key=True)
    year = Column(Integer)
    model = Column(String(255))
    vehicle_class = Column(String(255))
    fuel_type = Column(String(255))
    smog_rating = Column(Integer)
    city_mpg  = Column(Float)
    hwy_mpg  = Column(Float)
    cmb_mpg = Column(Float)
    Greenhouse_gas_score = Column(Integer)
    smartway = Column(String(255))

In [ ]:
c = 0
maxRows = 10000
start_time = datetime.datetime.now()
for index, row in final_mpg_df.iterrows():
    record = Fuel()
    record.year = row['year']
    record.model = row['model']
    record.fuel_type = row['fuel_type']
    record.vehicle_class = row['vehicle_class']
    record.smog_rating = row['smog_rating']
    record.city_mpg = row['city_mpg']
    record.hwy_mpg = row['hwy_mpg']
    record.cmb_mpg = row['cmb_mpg']
    record.Greenhouse_gas_score = row['Greenhouse_gas_score']
    record.smartway = row['smartway']
    
    session.add(record)
    session.commit()
    c = c + 1
    if c >= maxRows: break

end_time = datetime.datetime.now()
print(end_time - start_time)

In [ ]:
#pd.read_sql_query('select * from Fuel', con=engine)

In [ ]:

password = 'Minucar123456'

# AWS connection setup. Declare username and endpoint piece.
user = 'minalCarData'
endpoint = 'cardb.ci3ptaygzvuw.us-east-2.rds.amazonaws.com'
args = f'ssl_ca= database/config/rds-ca-2015-us-east-2-root.pem'

# AWS username and password.
rds_connection_string = f"{user}:{password}@{endpoint}/fuel_economy_db?{args}"
print(rds_connection_string)

In [ ]:
engine = create_engine(f'mysql://{rds_connection_string}')

conn = engine.connect()
session = Session(engine)
print("connected to AWS")


# Relect the existing database into a new model.

Base = automap_base()

# Reflect the table.

Base.prepare(engine, reflect=True)

# Save a reference to the ranks table as "Ranks".

Ranks = Base.classes.ranks